<a target="_blank" href="https://colab.research.google.com/github/motherduckdb/sql-tutorial">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 2. Learn to quack SQL with DuckDB: Group by, Joins and Subqueries

To start off, install the latest version of `duckdb` and `magic-duckdb` to run this notebook.

In [2]:
!pip install --upgrade duckdb magic-duckdb --quiet
%load_ext magic_duckdb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Example Tables
Let's start with two datasets:

- [birds.csv](https://raw.githubusercontent.com/motherduckdb/sql-tutorial/main/data/birds.csv): a list of measurements of individuals of different bird species
- [ducks.csv](https://raw.githubusercontent.com/motherduckdb/sql-tutorial/main/data/ducks.csv): a list of scientific names of bird species that are ducks

To download the datasets directly from GitHub, run:

In [3]:
!wget https://raw.githubusercontent.com/motherduckdb/sql-tutorial/main/data/birds.csv -q
!wget https://raw.githubusercontent.com/motherduckdb/sql-tutorial/main/data/ducks.csv -q
!wget https://raw.githubusercontent.com/motherduckdb/sql-tutorial/main/answers/answers_2.zip -q 
!unzip -o answers_2.zip -d answers 

Archive:  answers_2.zip
  inflating: answers/answer_2.01.sql  
  inflating: answers/answer_2.02.sql  
  inflating: answers/answer_2.03.sql  
  inflating: answers/answer_2.04.sql  
  inflating: answers/answer_2.05.sql  
  inflating: answers/answer_2.06.sql  
  inflating: answers/answer_2.07.sql  
  inflating: answers/answer_2.08.sql  
  inflating: answers/answer_2.09.sql  
  inflating: answers/answer_2.10.sql  
  inflating: answers/answer_2.11.sql  
  inflating: answers/answer_2.12.sql  
  inflating: answers/answer_2.13.sql  
  inflating: answers/answer_2.14.sql  


To create the tables in your database, run:

In [4]:
%%dql
CREATE TABLE birds AS SELECT * FROM read_csv('birds.csv');
CREATE TABLE ducks AS SELECT * FROM read_csv('ducks.csv');

,Count
0,139


To begin understanding the data contained in these tables, you can run:

In [5]:
%%dql
SUMMARIZE birds;

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,column00,BIGINT,0,90370,86041,45185.0,26088.004925635843,22592,45185,67778,90371,0.00
1,Avibase_ID,VARCHAR,AVIBASE-000E026B,AVIBASE-FFF63987,11083,None,None,None,None,None,90371,0.00
2,Species_Common_Name,VARCHAR,Abbott's babbler,ʻōʻū,8753,None,None,None,None,None,90371,19.24
3,Species1_BirdLife,VARCHAR,Abeillia abeillei,Zosterornis whiteheadi,11374,None,None,None,None,None,90371,0.37
4,Species2_eBird,VARCHAR,Abeillia abeillei,Zosterornis whiteheadi,10748,None,None,None,None,None,90371,0.40
5,eBird_species_group,VARCHAR,Abeillia abeillei,Zosterornis whiteheadi,11589,None,None,None,None,None,90371,0.20
6,Species3_BirdTree,VARCHAR,Abeillia abeillei,Zosterops xanthochroa,9200,None,None,None,None,None,90371,0.08
7,Data_type,DOUBLE,1.0,2.0,2,1.1574872250610975,0.3642615447315967,1.0,1.0,1.0,90371,0.39
8,Source,VARCHAR,AIM,ZRC,67,None,None,None,None,None,90371,0.40
9,Specimen_number,VARCHAR,"""1925.12.24.246B""",y3.5.12.1435,108176,None,None,None,None,None,90371,18.34


In [6]:
%%dql
SUMMARIZE ducks;

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,name,VARCHAR,African black duck,Yellow-billed teal,140,None,None,None,None,None,139,0.0
1,genus,VARCHAR,Aix galericulata,Thalassornis leuconotus,118,None,None,None,None,None,139,0.0
2,author,VARCHAR,A. Wilson,Wied-Neuwied,43,None,None,None,None,None,139,0.0
3,year,VARCHAR,1758,H. Fleming,57,None,None,None,None,None,139,0.0
4,extinct,BIGINT,0,1935,3,13.971223021582734,164.12040218163463,0,0,0,139,0.0


**Exercise 2.01**

Create a new table `birds_measurements` from the file `birds.csv` (this file contains the names and measurements of individuals from over 10k bird species).

In [7]:
%%dql
CREATE TABLE birds_measurements AS 
SELECT * FROM read_csv('birds.csv')

,Count
0,90371


In [8]:
# Uncomment and run to show solution
!cat ./answers/answer_2.01.sql

-- Create a new table `birds_measurements` from the file `birds.csv`, 
-- which contains the names and measurements of individuals from over 10k bird species.
CREATE TABLE birds_measurements AS 
    SELECT * FROM read_csv('birds.csv');

**Exercise 2.02**

Create a new table `ducks_species` from the file `ducks.csv` (this file contains species names and common names of ducks).

In [9]:
%%dql
CREATE TABLE ducks_species AS 
SELECT * FROM read_csv('ducks.csv')

,Count
0,139


In [10]:
# Uncomment and run to show solution
!cat ./answers/answer_2.02.sql

-- Create a new table `ducks_species` from the file `ducks.csv`, 
-- which contains species names and common names of ducks.
CREATE TABLE ducks_species AS 
    SELECT * FROM read_csv('ducks.csv');

## 1. Aggregate Functions

The functions we saw previously when building calculated columns operated on each row of the table individually. 
In contrast, <a href="https://duckdb.org/docs/sql/aggregates.html" target="_blank">aggregate functions</a> summarize many rows of the table. 
By default, they will summarize all rows (stay tuned though!). 
For example, let's find the minimum and maximum `Beak_Width` of any bird in the dataset.

In [11]:
%%dql
SELECT 
    MIN(Beak_Width) AS Min_Beak_Width,
    MAX(Beak_Width) AS Max_Beak_Width
FROM birds;

,Min_Beak_Width,Max_Beak_Width
0,0.7,94.2


However, aggregating an entire table all the way up to just a single row isn't always what we are looking for. 
Next, we will use the `GROUP BY` clause to apply aggregate functions to groups of rows instead of all rows.



## 2. Group Rows (GROUP BY Clause)


To group the rows based on a specific column (or columns) and perform <a href="https://duckdb.org/docs/sql/aggregates.html" target="_blank">aggregate functions</a>, you can use the `GROUP BY` clause. For example, if you want to group the birds by their species name and calculate the average `Beak_Width`, `Beak_Depth` and `Beak_Length_Culmen` for each group, you can run this query:

In [12]:
%%dql
SELECT
    Species_Common_Name,
    AVG(Beak_Width) AS Avg_Beak_Width,
    AVG(Beak_Depth) AS Avg_Beak_Depth,
    AVG(Beak_Length_Culmen) AS Avg_Beak_Length_Culmen
FROM birds
GROUP BY Species_Common_Name;

,Species_Common_Name,Avg_Beak_Width,Avg_Beak_Depth,Avg_Beak_Length_Culmen
0,Ovambo sparrowhawk,7.540000,10.260000,20.500000
1,Bank myna,5.516667,6.700000,24.350000
2,Pitcairn reed warbler,4.180000,3.960000,20.440000
3,Common iora,4.271429,4.471429,17.028571
4,Pectoral sparrow,5.510000,7.248276,14.654545
...,...,...,...,...
7988,Pearl-bellied white-eye,3.500000,4.400000,15.600000
7989,Mayotte white-eye,2.720000,2.980000,12.500000
7990,Réunion olive white-eye,2.550000,2.575000,13.900000
7991,Pemba white-eye,2.980000,3.040000,11.460000


This command groups the rows by the `Species_Common_Name` column and calculates the average `Beak_Width`, `Beak_Depth` and `Beak_Length_Culmen` for the individuals in each bird species group.

Multiple columns can be included within a `GROUP BY` clause, separated by commas. 
In this example, we measure the maximum `wing_length` by `Country_WRI` and `Source`.
This example shows that these columns do not have to be hierarchically related - the `GROUP BY` will show all combinations of data in those columns.

In [13]:
%%dql 
SELECT 
    Country_WRI,
    Source,
    MAX(wing_length)
FROM birds 
GROUP BY
    Country_WRI,
    Source

,Country_WRI,Source,max(wing_length)
0,Syria,NHMUK,486.0
1,Germany,ZFMK,614.0
2,India,MCZ,124.0
3,Samoa,NHMUK,242.0
4,None,MUSM,127.6
...,...,...,...
1059,Mauritania,NHMUK,226.0
1060,Yemen,SNSD,75.0
1061,Gambia,ZFMK,171.0
1062,Morocco,AMNH,281.0


**Exercise 2.03**


Run a query that gets the average `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` for all birds.

In [14]:
%%dql 
SELECT 
    AVG(beak_length_culmen),
    AVG(wing_length),
    AVG(tail_length)
FROM birds 


,avg(beak_length_culmen),avg(wing_length),avg(tail_length)
0,25.331714,111.780419,81.531344


In [15]:
# Uncomment and run to show solution
!cat ./answers/answer_2.03.sql

-- Run a query that gets the average `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` for all birds.
SELECT 
    AVG(Beak_Length_Culmen),
    AVG(Wing_Length),
    AVG(Tail_Length)
FROM birds

**Exercise 2.04**

Run a query that finds the average `Tail_Length` by `Species_Common_Name` and by `Country_WRI`.

In [16]:
%%dql 
SELECT
    species_common_name,
    country_wri,
    AVG(tail_length)
FROM birds
GROUP BY 
    species_common_name,
    country_wri

,Species_Common_Name,Country_WRI,avg(tail_length)
0,Brown thornbill,Australia,43.375000
1,Shikra,Sri Lanka,134.750000
2,Cooper's hawk,Canada,184.000000
3,Black sparrowhawk,Mozambique,210.000000
4,Eurasian sparrowhawk,United Kingdom,160.666667
...,...,...,...
20084,Abyssinian white-eye,United Republic of Tanzania,37.000000
20085,Malagasy white-eye,Madagascar,42.000000
20086,Christmas white-eye,None,56.000000
20087,Rota white-eye,United States of America,38.000000


In [17]:
# Uncomment and run to show solution
!cat ./answers/answer_2.04.sql

-- Run a query that finds the average `Tail_Length` by `Species_Common_Name` and by `Country_WRI`.
SELECT 
    AVG(Tail_Length)
FROM birds
GROUP BY
    Species_Common_Name,
    Country_WRI

### Getting the 95<sup>th</sup> percentile of a column value

We've used `GROUP BY` to group by a certain column, and used an aggregate function to combine other columns in our query, for instance, by taking the average. But, what if we want to get the 95<sup>th</sup> percentile of a column value? DuckDB has a built-in aggregate function for that too! For instance, to get the 95<sup>th</sup> percentile value of the `Beak_Length_Culmen` of all birds, run:

In [18]:
%%dql
SELECT 
    QUANTILE_CONT(Beak_Length_Culmen, 0.95)
FROM birds;

,"quantile_cont(Beak_Length_Culmen, 0.95)"
0,59.3


**Exercise 2.05**

Run a query that gets the 95<sup>th</sup> percentile and 99<sup>th</sup> percentile of `Beak_Length_Culmen` for all birds.

In [19]:
%%dql
SELECT 
    QUANTILE_CONT(Beak_Length_Culmen, 0.95),
    QUANTILE_CONT(beak_length_culmen, 0.99)
FROM birds;

,"quantile_cont(Beak_Length_Culmen, 0.95)","quantile_cont(beak_length_culmen, 0.99)"
0,59.3,122.2


In [20]:
# Uncomment and run to show solution
!cat ./answers/answer_2.05.sql

-- Run a query that gets the 95<sup>th</sup> percentile and 99<sup>th</sup> percentile of `Beak_Length_Culmen` for all birds.
SELECT 
    QUANTILE_CONT(Beak_Length_Culmen, 0.95),
    QUANTILE_CONT(Beak_Length_Culmen, 0.99)
FROM birds;

**Exercise 2.06**

Run a query that gets the 99<sup>th</sup> percentile of `Wing_Length` by `Species_Common_Name`.

In [21]:
%%dql
SELECT
    species_common_name,
    QUANTILE_CONT(wing_length, 0.99)
FROM birds
GROUP BY species_common_name

,Species_Common_Name,"quantile_cont(wing_length, 0.99)"
0,Ovambo sparrowhawk,241.84
1,Bank myna,123.95
2,Pitcairn reed warbler,81.84
3,Common iora,65.74
4,Pectoral sparrow,79.44
...,...,...
7988,Yellow-browed shrike-vireo,74.97
7989,Slender-billed xenops,64.50
7990,Sri Lanka thrush,124.97
7991,Large Lifou white-eye,73.84


In [22]:
# Uncomment and run to show solution
!cat ./answers/answer_2.06.sql

-- Run a query that gets the 99<sup>th</sup> percentile of `Wing_Length` by `Species_Common_Name`. 
SELECT 
    Species_Common_Name,
    QUANTILE_CONT(Wing_Length, 0.99),
FROM birds
GROUP BY
    Species_Common_Name;

## 3. Understanding SQL Joins

### INNER JOIN (JOIN)
In SQL, a Join operation allows you to combine rows from two or more tables based on a related column between them. This is incredibly useful when you need to pull together related information that is stored in different tables.

Let's combine the `birds` and `ducks` tables to find the `Beak_Length_Culmen` of all birds that are ducks. To do this, we'll use a SQL Join operation. Specifically, we'll use an `INNER JOIN`, which combines rows from both tables only when there is a match in the `Species_Common_Name` column.

In [23]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    ducks.author
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name;

,id,Species_Common_Name,Beak_Length_Culmen,author
0,2656,Egyptian goose,53.0,Linnaeus
1,2657,Egyptian goose,45.2,Linnaeus
2,2658,Egyptian goose,45.2,Linnaeus
3,2659,Egyptian goose,48.0,Linnaeus
4,2660,Egyptian goose,62.7,Linnaeus
...,...,...,...,...
657,81213,White-backed duck,45.0,Eyton
658,81214,White-backed duck,43.6,Eyton
659,81215,White-backed duck,41.0,Eyton
660,81216,White-backed duck,40.4,Eyton


### Step-by-Step Explanation
Let's break down the SQL query step by step:

`SELECT birds.column00 as id, birds.Species_Common_Name, birds.Beak_Length_Culmen, ducks.author`: We're selecting the species id, name and beak length from the `birds` table, and the duck species author from the `ducks` table.

Up until now, we haven't needed to specify which table a column is coming from since we have been working with just one table! 
Now we prefix column names with the name of the table they come from. 
As a note, this is not required if the column names in the two tables are completely different from one another, but it is a good best practice. 

`FROM birds`: We're starting with the `birds` table.

`INNER JOIN ducks ON birds.Species_Common_Name = ducks.name`: We're joining the birds table to the ducks table where the species' common name matches in both tables. 
We are using table prefixes again for clarity.

### INNER JOIN Gotchas

**NOTE:** When using an `INNER JOIN`, we only show output rows where there are matching values in both tables. 
This has dramatically reduced the number of output rows since now we are only looking at ducks!

**NOTE:** If a join between 2 tables results in multiple matches, all matches will be returned. 
This can mean that your result can actually return **more** rows after a join, in some cases!
(Imagine that we had messy data in `ducks.csv`, and one species mistakenly had multiple authors. We would have 1 row in our result for each author.)

**NOTE:** `INNER JOIN` is the default kind of join in SQL. So if you see a query that just says `... table1 JOIN table2 ...`, then it is using an `INNER JOIN`!
It is common practice to omit `INNER`. 

**NOTE:** It is possible to join on multiple columns. 
For example, imagine wanting to connect two tables by matching both a first name column and last name column. 
Inequality conditions are also possible (as we will see later!).

**Exercise 2.07**

Run a query that gets the name, `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` of birds that are ducks.

In [24]:
%%dql
SELECT
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    birds.wing_length,
    birds.tail_length
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name;

,Species_Common_Name,Beak_Length_Culmen,Wing_Length,Tail_Length
0,Egyptian goose,53.0,403.0,140.0
1,Egyptian goose,45.2,392.0,148.0
2,Egyptian goose,45.2,360.0,113.0
3,Egyptian goose,48.0,350.0,112.0
4,Egyptian goose,62.7,344.0,138.0
...,...,...,...,...
657,White-backed duck,45.0,NaN,46.0
658,White-backed duck,43.6,162.0,38.0
659,White-backed duck,41.0,174.0,47.0
660,White-backed duck,40.4,161.0,42.0


In [25]:
# Uncomment and run to show solution
!cat ./answers/answer_2.07.sql

-- Run a query that gets the `Species_Common_Name`, `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` of birds that are ducks.
SELECT 
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    birds.Wing_Length,
    birds.Tail_Length
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name;

**Exercise 2.08**

Let's run a similar query, but group the ducks by species. Run a query that gets the `Species_Common_Name`, _average_ `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` of birds that are ducks, and sort the results by `Species_Common_Name`.

In [26]:
%%dql
SELECT
    birds.Species_Common_Name,
    AVG(birds.Beak_Length_Culmen),
    AVG(birds.wing_length),
    AVG(birds.tail_length)
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
GROUP BY birds.species_common_name
ORDER BY birds.species_common_name;

,Species_Common_Name,avg(birds.Beak_Length_Culmen),avg(birds.wing_length),avg(birds.tail_length)
0,African black duck,48.380000,247.400000,112.400000
1,American black duck,59.400000,267.200000,73.500000
2,Andean teal,40.080000,224.200000,82.200000
3,Auckland teal,44.800000,134.800000,75.200000
4,Australian shelduck,51.050000,353.500000,106.000000
...,...,...,...,...
100,White-headed duck,48.000000,148.400000,88.200000
101,Wood duck,38.650000,220.500000,106.833333
102,Yellow-billed duck,53.285714,240.571429,96.000000
103,Yellow-billed pintail,34.300000,205.600000,89.400000


In [27]:
# Uncomment and run to show solution
!cat ./answers/answer_2.08.sql

-- Let's run a similar query, but group the ducks by species. 
-- Run a query that gets the `Species_Common_Name`, _average_ `Beak_Length_Culmen`, `Wing_Length` and `Tail_Length` of
-- birds that are ducks, and sort the results by `Species_Common_Name`.
SELECT 
    birds.Species_Common_Name,
    AVG(birds.Beak_Length_Culmen),
    AVG(birds.Wing_Length),
    AVG(birds.Tail_Length)
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
GROUP BY 
    birds.Species_Common_Name
ORDER BY 
    birds.Species_Common_Name;

### LEFT OUTER JOIN (LEFT JOIN)

A `LEFT OUTER JOIN` will keep all rows from the `LEFT` table in the join (the table before the `LEFT JOIN` keywords), even if there is not a match in the table on the right.
Any rows that do not have a match in the right table will have the value `NULL` for all columns from the right table. 
`NULL` is the missing value indicator in SQL. 

This can be useful when adding optional details.
For example, in our situtation, ducks will have an author, but all other birds will not.

In [28]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    ducks.author
FROM birds
LEFT JOIN ducks ON birds.Species_Common_Name = ducks.name;

,id,Species_Common_Name,Beak_Length_Culmen,author
0,0,Emerald-chinned hummingbird,14.8,None
1,1,Emerald-chinned hummingbird,13.8,None
2,2,Emerald-chinned hummingbird,11.8,None
3,3,Emerald-chinned hummingbird,11.8,None
4,4,Emerald-chinned hummingbird,12.8,None
...,...,...,...,...
90366,81213,White-backed duck,45.0,Eyton
90367,81214,White-backed duck,43.6,Eyton
90368,81215,White-backed duck,41.0,Eyton
90369,81216,White-backed duck,40.4,Eyton


Notice how the `LEFT JOIN` query has 90371 rows in the result (the same number of rows as the `birds` table), and the `INNER JOIN` query only had 662 rows. 

The `author` column contains the Python missing value indicator of `None`, which is equivalent to SQL's `NULL`. 

### LEFT JOIN Gotchas

**NOTE:** A `LEFT JOIN` usually, but not always, will result in the same number of rows as the left table. 
Cases where this is not true include:
* Duplicates in the columns that are being joined in the right table
* A `WHERE` clause that filters the result

**Exercise 2.09**

Modify the `LEFT JOIN` query above to filter to only rows that are **NOT** ducks. 

Hint: In Python (like in SQL), nothing equals None! 
Just like in Python, we use the `IS` keyword to check if a value is missing.

In [29]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    ducks.author
FROM birds
LEFT JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE ducks.author is NULL ;

,id,Species_Common_Name,Beak_Length_Culmen,author
0,0,Emerald-chinned hummingbird,14.8,None
1,1,Emerald-chinned hummingbird,13.8,None
2,2,Emerald-chinned hummingbird,11.8,None
3,3,Emerald-chinned hummingbird,11.8,None
4,4,Emerald-chinned hummingbird,12.8,None
...,...,...,...,...
89704,90366,None,15.3,None
89705,90367,None,17.3,None
89706,90368,None,18.9,None
89707,90369,None,17.2,None


In [30]:
# Uncomment and run to show solution
!cat ./answers/answer_2.09.sql

-- Modify the `LEFT JOIN` query above to filter to only rows that are **NOT** ducks. 

-- Hint: In Python (like in SQL), nothing equals None! 
-- Just like in Python, we use the `IS` keyword to check if a value is missing.
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen,
    ducks.author
FROM birds
LEFT JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE 
    ducks.name IS NULL;

## 3. Subqueries

### What is a Subquery?

A subquery, also known as a nested query, is a query within another SQL query. It's like a query inside a query! Subqueries are used to perform operations that require multiple steps, such as filtering data based on a complex condition or aggregating data before using it in the main query. In other words, instead of creating multiple new tables as intermediate steps, you can define these steps within the scope of a larger query.

### Types of Subquery

A Subquery can return a single value (one row and one column), an entire column of values, or a table of values. 
These each can be used in the location within a query where a static value, column, or table would otherwise be.

### Using Subqueries in DuckDB

Let's start by looking at our previously example query to understand how subqueries work in DuckDB.

#### Finding the top `Beak_Length_Culmen`

Suppose we want to find the _individual_ ducks with the largest `Beak_Length_Culmen`. We can use a subquery to calculate the 99<sup>th</sup> percentile of `Beak_Length_Culmen` first, and then use that result in our main query:

In [31]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Beak_Length_Culmen
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE birds.Beak_Length_Culmen > (
    SELECT QUANTILE_CONT(birds.Beak_Length_Culmen, 0.99)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)
ORDER BY birds.Beak_Length_Culmen DESC;

,id,Species_Common_Name,Beak_Length_Culmen
0,48763,Common merganser,79.0
1,48764,Common merganser,78.7
2,54085,Knob-billed duck,74.5
3,10784,Muscovy duck,74.0
4,46501,Pink-eared duck,73.4
5,75575,Common eider,73.1
6,46502,Pink-eared duck,72.2


In this example, the subquery (`SELECT QUANTILE_CONT(birds.Beak_Length_Culmen, 0.99) FROM birds INNER JOIN ducks ON birds.Species_Common_Name = ducks.name`) calculates the 99<sup>th</sup> percentile of beak length for all birds that are ducks. The main query then selects the names and beak measurements of individual ducks who have a beak length above this value.

**Exercise 2.10**


Find the duck species that have a `Wing_Length` larger than the 99<sup>th</sup> percentile of all ducks.

In [32]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.wing_length
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE birds.wing_length > (
    SELECT QUANTILE_CONT(birds.wing_length, 0.99)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)

,id,Species_Common_Name,Wing_Length
0,10782,Muscovy duck,397.0
1,10784,Muscovy duck,414.0
2,10787,Muscovy duck,405.0
3,2656,Egyptian goose,403.0
4,2657,Egyptian goose,392.0
5,79479,Ruddy shelduck,383.0
6,79501,Paradise shelduck,380.0


In [33]:
# Uncomment and run to show solution
!cat ./answers/answer_2.10.sql

-- Find the duck species that have a `Wing_Length` larger than the 99<sup>th</sup> percentile of all ducks.
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Wing_Length
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE birds.Wing_Length > (
    SELECT QUANTILE_CONT(birds.Wing_Length, 0.99)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)
ORDER BY birds.Wing_Length DESC;

**Exercise 2.11**

Can you find any duck species that have both a `Wing_Length` _and_ `Beak_Length_Culmen` larger than the 95<sup>th</sup> percentile of all duck species?

In [34]:
%%dql
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.wing_length,
    birds.beak_length_culmen
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE birds.wing_length > (
    SELECT QUANTILE_CONT(birds.wing_length, 0.95)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
) AND
birds.beak_length_culmen > (
    SELECT QUANTILE_CONT(birds.beak_length_culmen, 0.95)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)

,id,Species_Common_Name,Wing_Length,Beak_Length_Culmen
0,10787,Muscovy duck,405.0,71.5
1,54085,Knob-billed duck,362.0,74.5
2,10785,Muscovy duck,361.0,66.2
3,10784,Muscovy duck,414.0,74.0
4,10782,Muscovy duck,397.0,70.4


In [35]:
# Uncomment and run to show solution
!cat ./answers/answer_2.11.sql

-- Can you find any duck species that have both a `Wing_Length` _and_ `Beak_Length_Culmen` 
-- larger than the 95<sup>th</sup> percentile of all duck species?
SELECT
    birds.column00 as id,
    birds.Species_Common_Name,
    birds.Wing_Length,
    birds.Beak_Length_Culmen
FROM birds
INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
WHERE birds.Wing_Length > (
    SELECT QUANTILE_CONT(birds.Wing_Length, 0.95)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)
AND birds.Beak_Length_Culmen > (
    SELECT QUANTILE_CONT(birds.Beak_Length_Culmen, 0.95)
    FROM birds 
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
)
ORDER BY birds.Wing_Length DESC;

**Exercise 2.12**

NOTE: This is extra credit!

Instead of individual ducks, find the duck species that _on average_ have a measured beak size that is larger than the 95<sup>th</sup> percentile of all ducks.

In [36]:
%%dql
DESCRIBE birds

,column_name,column_type,null,key,default,extra
0,column00,BIGINT,YES,None,None,None
1,Avibase_ID,VARCHAR,YES,None,None,None
2,Species_Common_Name,VARCHAR,YES,None,None,None
3,Species1_BirdLife,VARCHAR,YES,None,None,None
4,Species2_eBird,VARCHAR,YES,None,None,None
5,eBird_species_group,VARCHAR,YES,None,None,None
6,Species3_BirdTree,VARCHAR,YES,None,None,None
7,Data_type,DOUBLE,YES,None,None,None
8,Source,VARCHAR,YES,None,None,None
9,Specimen_number,VARCHAR,YES,None,None,None


In [44]:
%%dql
SELECT *
FROM (SELECT
        birds.Species_Common_Name,
        AVG(birds.beak_length_culmen) AS avg_beak_length_culmen
    FROM birds
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
    GROUP BY birds.species_common_name
)
WHERE 
    avg_beak_length_culmen > (
        SELECT QUANTILE_CONT(birds.beak_length_culmen, 0.95)
        FROM birds 
        INNER JOIN ducks ON birds.Species_Common_Name = ducks.name)


,Species_Common_Name,avg_beak_length_culmen
0,Muscovy duck,65.933333
1,Common merganser,69.422222
2,Pink-eared duck,67.616667


In [38]:
# Uncomment and run to show solution
!cat ./answers/answer_2.12.sql

-- Instead of individual ducks, find the duck species that _on average_ have a measured beak size 
-- that is larger than the 95<sup>th</sup> percentile of all ducks.
SELECT * 
FROM (
    SELECT 
        birds.Species_Common_Name,
        AVG(birds.Beak_Length_Culmen) as Avg_Beak_Length_Culmen
    FROM birds
    INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
    GROUP BY 
        birds.Species_Common_Name
)
WHERE 
    Avg_Beak_Length_Culmen > (
        SELECT QUANTILE_CONT(birds.Beak_Length_Culmen, 0.95)
        FROM birds 
        INNER JOIN ducks ON birds.Species_Common_Name = ducks.name
    )
;

#### Using the WITH Clause

The `WITH` clause is an alternative to a subquery that has 2 key advantages: it can increase readability, and it allows for reusability. 
The technical term for a `WITH` clause is a Common Table Expression (abbreviated CTE), which describes how it can be reusable.

Now, let's see how we can use the `WITH` clause to make our queries more readable. Suppose we want to find the individual ducks that have a beak length above the 99<sup>th</sup> percentile of duck beaks. Here's how we can do it using the `WITH` clause:

In [45]:
%%dql
WITH
    duck_beaks AS (
        SELECT
            column00 as id,
            Species_Common_Name,
            Beak_Length_Culmen
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
    ),

    pc99_beak_len AS (
        SELECT QUANTILE_CONT(Beak_Length_Culmen, 0.99) AS Top_Beak_Length 
        FROM duck_beaks
    )

SELECT
    duck_beaks.id,
    duck_beaks.Species_Common_Name,
    duck_beaks.Beak_Length_Culmen
FROM duck_beaks
INNER JOIN pc99_beak_len ON duck_beaks.Beak_Length_Culmen > pc99_beak_len.Top_Beak_Length
ORDER BY duck_beaks.Beak_Length_Culmen DESC;

,id,Species_Common_Name,Beak_Length_Culmen
0,48763,Common merganser,79.0
1,48764,Common merganser,78.7
2,54085,Knob-billed duck,74.5
3,10784,Muscovy duck,74.0
4,46501,Pink-eared duck,73.4
5,75575,Common eider,73.1
6,46502,Pink-eared duck,72.2


In this example, the `WITH` clause creates two temporary result sets called `duck_beaks` and `pc99_beak_len`. The main query then selects the names and beak measurements of ducks with `Beak_Length_Culmen` above the top 99<sup>th</sup> percentile beak length.

**Exercise 2.13**

Find the duck species that have an average `Wing_Length` larger than the 95<sup>th</sup> percentile of all duck species.

In [57]:
%%dql
WITH
    duck_beaks AS (
        SELECT
            Species_Common_Name,
            AVG(wing_length) as avg_wing_length
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
        GROUP BY Species_Common_Name
    ),

    pc95_wing_len AS (
        SELECT QUANTILE_CONT(wing_length, 0.95) AS top_wing_length 
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
    )

SELECT
    duck_beaks.Species_Common_Name,
    duck_beaks.avg_wing_length
FROM duck_beaks
INNER JOIN pc95_wing_len ON duck_beaks.avg_wing_length > pc95_wing_len.top_wing_length
ORDER BY duck_beaks.avg_wing_length DESC;

,Species_Common_Name,avg_wing_length
0,Nene,364.0
1,Egyptian goose,356.0
2,Ruddy shelduck,355.5
3,Australian shelduck,353.5
4,Paradise shelduck,351.2
5,South African shelduck,345.2
6,Muscovy duck,345.0


In [48]:
# Uncomment and run to show solution
!cat ./answers/answer_2.13.sql

-- Find the duck species that have an average `Wing_Length` larger than the 95<sup>th</sup> percentile of all duck species.
WITH
    duck_wings AS (
        SELECT
            Species_Common_Name,
            AVG(Wing_Length) as Avg_Wing_Length
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
        GROUP BY 
            Species_Common_Name
    ),

    pc99_beak_len AS (
        SELECT QUANTILE_CONT(Wing_Length, 0.95) AS Top_Wing_Length 
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
    )

SELECT
    duck_wings.Species_Common_Name,
    duck_wings.Avg_Wing_Length
FROM duck_wings
INNER JOIN pc99_beak_len ON duck_wings.Avg_Wing_Length > pc99_beak_len.Top_Wing_Length
ORDER BY duck_wings.Avg_Wing_Length DESC;

**Exercise 2.14**

What about the duck species that have both a `Wing_Length` _or_ `Beak_Length_Culmen` larger than the 95sup>th</sup> percentile of all duck species?

In [68]:
%%dql
WITH
    duck AS (
        SELECT
            Species_Common_Name,
            AVG(wing_length) as avg_wing_length,
            AVG(beak_length_culmen) as avg_beak_length
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
        GROUP BY species_common_name
    ),

    pc95 AS (
        SELECT 
            QUANTILE_CONT(wing_length, 0.95) AS top_wing_length,
            QUANTILE_CONT(Beak_Length_Culmen, 0.95) AS Top_Beak_Length
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
    )

SELECT
    duck.Species_Common_Name,
    duck.avg_wing_length
FROM duck
INNER JOIN pc95 ON duck.avg_wing_length > pc95.top_wing_length
OR duck.avg_beak_length > pc95.top_beak_length
ORDER BY duck.avg_wing_length DESC;

,Species_Common_Name,avg_wing_length
0,Nene,364.000000
1,Egyptian goose,356.000000
2,Ruddy shelduck,355.500000
3,Australian shelduck,353.500000
4,Paradise shelduck,351.200000
5,South African shelduck,345.200000
6,Muscovy duck,345.000000
7,Common merganser,270.769231
8,Pink-eared duck,183.666667


In [63]:
# Uncomment and run to show solution
!cat ./answers/answer_2.14.sql

-- What about the duck species that have a `Wing_Length` _or_ `Beak_Length_Culmen` 
-- larger than the 95<sup>th</sup> percentile of all duck species?
WITH
    duck_wings AS (
        SELECT
            Species_Common_Name,
            AVG(Wing_Length) as Avg_Wing_Length,
            AVG(Beak_Length_Culmen) as Avg_Beak_Length
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
        GROUP BY 
            Species_Common_Name
    ),

    pc95 AS (
        SELECT 
            QUANTILE_CONT(Wing_Length, 0.95) AS Top_Wing_Length,
            QUANTILE_CONT(Beak_Length_Culmen, 0.95) AS Top_Beak_Length,
        FROM birds
        INNER JOIN ducks ON name = Species_Common_Name
    )


SELECT
    duck_wings.Species_Common_Name,
    duck_wings.Avg_Wing_Length
FROM duck_wings
INNER JOIN pc95 
    ON duck_wings.Avg_Wing_Length > pc95.Top_Wing_Length
    OR duck_wings.Avg_Beak_Length > pc95.Top_Beak_Length
ORDER BY duck_wings.Avg_Wing_Length DESC;
